In [ ]:
import numpy as np
from tqdm import tqdm_notebook

from sdcdup.utils import overlap_tag_pairs
from sdcdup.utils import overlap_tag_maps
from sdcdup.utils import get_overlap_matches
from sdcdup.utils import load_duplicate_truth
from sdcdup.features import SDCImageContainer

%reload_ext autoreload
%autoreload 2

In [ ]:
matches_files = [
    'matches_bmh32_0.9_offset.csv',
    'matches_bmh96_0.9_offset.csv', 
    'matches_bmh32_0.8.csv',
    'matches_bmh96_0.8.csv', 
]
sdcic = SDCImageContainer()
sdcic.matches = get_overlap_matches(matches_files)

In [ ]:
overlap_image_maps = sdcic.load_image_overlap_properties(matches_files, score_types=['shp'])
print(len(overlap_image_maps))

In [ ]:
dup_truth = load_duplicate_truth()
print(len(dup_truth))

## Find overlaps with ships

In [ ]:
untested_image_pairs_with_ship_masks = []
for (img1_id, img2_id), overlap_maps in tqdm_notebook(overlap_image_maps.items()):
    # TODO: Find out which remaining tile pairs have masks but aren't in dup_truth.

    mask1 = sdcic.lbl_metrics['shp'][img1_id]
    mask2 = sdcic.lbl_metrics['shp'][img2_id]
    
    has_mask1 = np.sum(mask1) > 0
    has_mask2 = np.sum(mask2) > 0

    if not (has_mask1 and has_mask2):
        continue

    for img1_overlap_tag in overlap_maps:
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            continue
        untested_image_pairs_with_ship_masks.append((img1_id, img2_id))
        break

len(untested_image_pairs_with_ship_masks)

In [ ]:
untested_overlaps_with_ship_masks = []
for (img1_id, img2_id), overlap_maps in tqdm_notebook(overlap_image_maps.items()):
    # TODO: Find out which remaining tile pairs have masks but aren't in dup_truth.

    mask1 = sdcic.lbl_metrics['shp'][img1_id]
    mask2 = sdcic.lbl_metrics['shp'][img2_id]
    
    has_mask1 = np.sum(mask1) > 0
    has_mask2 = np.sum(mask2) > 0

    if not (has_mask1 and has_mask2):
        continue

    for img1_overlap_tag in overlap_maps:
        if (img1_id, img2_id, img1_overlap_tag) in dup_truth:
            continue

        mask1_slice_total = np.sum(mask1[overlap_tag_maps[img1_overlap_tag]])
        mask2_slice_total = np.sum(mask2[overlap_tag_maps[overlap_tag_pairs[img1_overlap_tag]]])

        if mask1_slice_total + mask2_slice_total < 1:
            continue

        untested_overlaps_with_ship_masks.append((img1_id, img2_id, img1_overlap_tag))

len(untested_overlaps_with_ship_masks)